In [1]:
import os
from tqdm import tqdm
from multiprocessing import Pool
from collections import defaultdict

In [2]:
# Read the data
with open('test.wp_target', 'r') as file:
    data = file.read()

print(f"Size of data:{len(data)}")
print(f"Data:{data[:100]}")

Size of data:50398417
Data:The wet marble floor pressed on his cheek like a thousand hands slapping his face frozen in time . S


In [3]:
data_list = data.split("\n")
print(f"size of data_list:{len(data_list)}")

size of data_list:15139


In [4]:
# Convert <newline> to \n
data_list = [x.replace("<newline>", "\n") for x in data_list]
# Append </BOT> and <EOT> to all the lines
data_list = ["</BOT>" + x + "<EOT>" for x in data_list]
print(data_list[0])

</BOT>The wet marble floor pressed on his cheek like a thousand hands slapping his face frozen in time . Smattering piss of rain ignored his indignant mumblings . His eyes fluttered . Pins and needs ran from finger to shoulder as he pushed back against the floor , contorting his aching body into a cross legged position . Last night was bad . He gathered that . His routine dullness of though crept inwards from the edges of his mind toward the black mist that veiled his most recent memories . He struggled to recall whatever he could n't recall but only for a moment before he decided it probably was n't worth the effort . 
 He glanced around the room for a few minutes before concluding that he probably did n't know where he was . His investigation was n't entirely fruitless , he discovered a mostly full bottle of vodka . It was cheap but would definitely get the job done . Taking a few swigs made it childishly easy to ignore that gigantic black cloud of fog blotting out whatever the hell 

In [5]:
# Split the data into word corpus for BPE procesing
data_corpus = [word for line in data_list for word in line.split()]
print(f"Size of data_corpus:{len(data_corpus)}")
print(f"Data_corpus:{data_corpus[:100]}")

Size of data_corpus:9777952
Data_corpus:['</BOT>The', 'wet', 'marble', 'floor', 'pressed', 'on', 'his', 'cheek', 'like', 'a', 'thousand', 'hands', 'slapping', 'his', 'face', 'frozen', 'in', 'time', '.', 'Smattering', 'piss', 'of', 'rain', 'ignored', 'his', 'indignant', 'mumblings', '.', 'His', 'eyes', 'fluttered', '.', 'Pins', 'and', 'needs', 'ran', 'from', 'finger', 'to', 'shoulder', 'as', 'he', 'pushed', 'back', 'against', 'the', 'floor', ',', 'contorting', 'his', 'aching', 'body', 'into', 'a', 'cross', 'legged', 'position', '.', 'Last', 'night', 'was', 'bad', '.', 'He', 'gathered', 'that', '.', 'His', 'routine', 'dullness', 'of', 'though', 'crept', 'inwards', 'from', 'the', 'edges', 'of', 'his', 'mind', 'toward', 'the', 'black', 'mist', 'that', 'veiled', 'his', 'most', 'recent', 'memories', '.', 'He', 'struggled', 'to', 'recall', 'whatever', 'he', 'could', "n't", 'recall']


In [6]:
# Initialize the vocabulary
special_tokens = {
    '</BOT>': 0,
    '</EOT>': 1
}
inv_special_tokens = { v:k for k,v in special_tokens.items() }
token_map = {}
token_map.update(special_tokens)
inv_token_map = {}
inv_token_map.update(inv_special_tokens)
bpe_codes = {}

In [7]:
# word splitting and counting
vocab = { '</BOT>' : 0, '</EOT>' : 1 }
for word in data_corpus:
    if word in special_tokens.keys():
        continue
    chars = list(word) + ['</w>']
    word_tuple = tuple(chars)
    vocab[word_tuple] = vocab.get(word_tuple, 0) + 1

In [8]:
# Collect unique symbols
symbols = set()
for word_tuple in vocab:
    symbols.update(word_tuple)

In [9]:
# Check the symbols
# There are many non-english symbols
print(symbols)

{'ö', 'Ｔ', '为', ';', 'к', 'ग', '’', 'ř', '角', 'र', '¯', 'ل', '¿', 'स', 'å', '̎', 'ｗ', '>', 'ג', 'l', '‘', 'Ï', '̷', '̪', '상', '̽', 'î', 'Δ', 'Ð', 'T', 'ờ', 'V', 'ͦ', 'د', 'ִ', 'a', 'M', '*', '̶', 'ف', '万', '̥', '∴', '－', '̨', 'F', '̤', ']', 'G', 'Î', 'À', 'ي', '용', 'ō', '̂', 'פ', '》', '-', 'p', 't', ')', 'Ԩ', 'غ', 'ק', '̊', '💦', '%', 'ͧ', '8', 'ఠ', 'е', '−', 'ù', 'x', '֏', '̉', 'े', 'Ｌ', 'な', '⬆', 'ह', '_', '么', '̚', 'β', 'آ', '̖', 'Y', 'μ', '😩', '̇', '÷', 'Ｘ', '̝', 'ლ', 'Ａ', '°', 'í', 'ث', '͛', 'ड', 'Q', 'α', '͝', '̸', 'ח', 'כ', '1', 'à', '🤔', '̵', 'Ü', '̈́', 'R', '̮', '̳', 'H', '̓', '的', '☹', '«', '𝘑', '͂', 'þ', '̘', '毛', 'а', '§', 'r', 'Ｙ', 'ك', 'й', '0', 'ソ', 'н', '̺', 'ी', '͟', 'ç', 'Ｃ', 'ñ', '{', '𝘴', 'ø', 'л', '͈', 'v', '‽', 'ô', '9', '̴', '💯', '</w>', '\\', 'ج', 'J', '¾', 'ط', '̭', 'у', 'E', 'ͩ', '7', '͇', 'ּ', 'ן', '̔', 'é', 'º', 'Ｇ', '《', 'Ś', '̜', '生', 'e', '3', '͗', 'т', 'э', 'ă', '»', '₴', '̼', 'ï', 'ã', '\u200b', 'u', 'ब', 'ȩ', '厦', 'ז', '†', '♦', '͎', '村', 'ى', 'ͭ', 'ָ', 

In [10]:
token_id = 2 # 0,1 are reserved for special tokens. 
print(type(token_map))
# If we wanted to do this dynamically we can do token_id = len(special_tokens)
for symbol in symbols:
    if symbol not in token_map:
        token_map[symbol] = token_id
        inv_token_map[token_id] = symbol
        token_id += 1

<class 'dict'>


In [11]:
inv_map = {v: k for k, v in token_map.items()}

In [12]:
print(token_map.keys())
print(inv_token_map.keys())

dict_keys(['</BOT>', '</EOT>', 'ö', 'Ｔ', '为', ';', 'к', 'ग', '’', 'ř', '角', 'र', '¯', 'ل', '¿', 'स', 'å', '̎', 'ｗ', '>', 'ג', 'l', '‘', 'Ï', '̷', '̪', '상', '̽', 'î', 'Δ', 'Ð', 'T', 'ờ', 'V', 'ͦ', 'د', 'ִ', 'a', 'M', '*', '̶', 'ف', '万', '̥', '∴', '－', '̨', 'F', '̤', ']', 'G', 'Î', 'À', 'ي', '용', 'ō', '̂', 'פ', '》', '-', 'p', 't', ')', 'Ԩ', 'غ', 'ק', '̊', '💦', '%', 'ͧ', '8', 'ఠ', 'е', '−', 'ù', 'x', '֏', '̉', 'े', 'Ｌ', 'な', '⬆', 'ह', '_', '么', '̚', 'β', 'آ', '̖', 'Y', 'μ', '😩', '̇', '÷', 'Ｘ', '̝', 'ლ', 'Ａ', '°', 'í', 'ث', '͛', 'ड', 'Q', 'α', '͝', '̸', 'ח', 'כ', '1', 'à', '🤔', '̵', 'Ü', '̈́', 'R', '̮', '̳', 'H', '̓', '的', '☹', '«', '𝘑', '͂', 'þ', '̘', '毛', 'а', '§', 'r', 'Ｙ', 'ك', 'й', '0', 'ソ', 'н', '̺', 'ी', '͟', 'ç', 'Ｃ', 'ñ', '{', '𝘴', 'ø', 'л', '͈', 'v', '‽', 'ô', '9', '̴', '💯', '</w>', '\\', 'ج', 'J', '¾', 'ط', '̭', 'у', 'E', 'ͩ', '7', '͇', 'ּ', 'ן', '̔', 'é', 'º', 'Ｇ', '《', 'Ś', '̜', '生', 'e', '3', '͗', 'т', 'э', 'ă', '»', '₴', '̼', 'ï', 'ã', '\u200b', 'u', 'ब', 'ȩ', '厦', 'ז', '†', 

In [13]:
# get_pair_counts function
def get_pair_counts(vocab: dict[tuple[str], int]) -> dict[tuple[str, str], int]:
    """
    Count the frequency of each pair of symbols in the vocabulary
    by iterating over all words in the vocabulary and counting
    """
    pairs = {}
    for word, freq in vocab.items():
        symbols = word
        for i in range(len(symbols)-1): # There's no pair for last symbol
            pair = (symbols[i], symbols[i+1])
            pairs[pair] = pairs.get(pair, 0) + freq
    return pairs

In [14]:
# merge_vocab function
from multiprocessing import Pool
def merge_vocab_primitive(pair: tuple[str, str], vocab:dict[tuple[str], int]) -> dict[tuple[str], int]:
    """
    Merge the most frequent pair in the vocabulary
    This is primitive version of merge_vocab using single thread
    to demonstrate algorithm
    """
    new_vocab = {}
    bigram = ''.join(pair) # join two symbols to create a bigram
    for word, freq in vocab.items():
        w = []
        i = 0
        while i < len(word)-1: # Reason for wile loop is inconsistent index combing
            # Merge pair if found
            if i < len(word) - 1 and word[i] == pair[0] and word[i+1] == pair[1]:
                w.append(bigram)
            else:
                w.append(word[i])
                i += 1
        new_vocab[tuple(w)] = freq
    return new_vocab

def process_word(args):
    pair, word_freq = args
    word, freq = word_freq
    bigram = ''.join(pair)
    w = []
    i = 0
    while i < len(word):
        if i < len(word) - 1 and word[i] == pair[0] and word[i + 1] == pair[1]:
            w.append(bigram)
            i += 2
        else:
            w.append(word[i])
            i += 1
    return tuple(w), freq

def merge_vocab(pair: tuple[str, str], vocab:dict[tuple[str], int]) -> dict[tuple[str], int]:
    """
    Parallel merge of all occurrences of the given pair in the vocabulary using multiprocessing.
    GLI is disabled in python3.13rc, but current version is python3.12, hence 
    concurrent.future is near equivalent to single-threaded version
    """
    with Pool() as pool:
        results = pool.map(process_word, [(pair, item) for item in vocab.items()])

    new_vocab = {word: freq for word, freq in results}
    return new_vocab


In [15]:
# BPE Merges
num_merges = 1000
bpe_codes = {}
for i in tqdm(range(num_merges)):
    pairs = get_pair_counts(vocab)
    # print(f"Best pair:{max(pairs, key=pairs.get)}")
    # print(f"BPE Codes:{bpe_codes}")
    if not pairs:
        break
    best_pair = max(pairs, key=pairs.get)
    vocab = merge_vocab(best_pair, vocab)
    bpe_codes[best_pair] = i
    new_symbol = ''.join(best_pair)
    if new_symbol not in token_map:
        token_map[new_symbol] = token_id
        inv_token_map[token_id] = new_symbol
        token_id += 1
    

100%|██████████| 1000/1000 [19:59<00:00,  1.20s/it]


In [16]:
# Save the BPE code, token map to observe the result
import json
with open('bpe_codes.json', 'w') as file:
    json.dump({f"[{' , '.join(word)}]" :code for word, code in bpe_codes}, 
              file,
              indent=4,
              ensure_ascii=False)

with open('token_map.json', 'w') as file:
    json.dump(token_map, file, indent=4, ensure_ascii=False)

In [17]:
# Encoding a sample sentence using BPE
sentence = data_list[0]
print(sentence)

</BOT>The wet marble floor pressed on his cheek like a thousand hands slapping his face frozen in time . Smattering piss of rain ignored his indignant mumblings . His eyes fluttered . Pins and needs ran from finger to shoulder as he pushed back against the floor , contorting his aching body into a cross legged position . Last night was bad . He gathered that . His routine dullness of though crept inwards from the edges of his mind toward the black mist that veiled his most recent memories . He struggled to recall whatever he could n't recall but only for a moment before he decided it probably was n't worth the effort . 
 He glanced around the room for a few minutes before concluding that he probably did n't know where he was . His investigation was n't entirely fruitless , he discovered a mostly full bottle of vodka . It was cheap but would definitely get the job done . Taking a few swigs made it childishly easy to ignore that gigantic black cloud of fog blotting out whatever the hell 

In [18]:
def _get_pairs(word: list[str]) -> set[tuple[str, str]]:
    """
    Get all pairs of symbols in a word
    """
    pairs = set()
    for i in range(len(word)-1):
        pairs.add((word[i], word[i+1]))

    return pairs

def apply_bpe(word: list[str]) -> list[str]:
    """
    Apply BPE to a list of symbols(a word)
    """
    word = word.copy()
    pairs = _get_pairs(word)
    while True:
        if not pairs:
            break
        # Find highest priority pair to merge
        min_pair = None
        min_rank = float('inf')
        for pair in pairs:
            if pair in bpe_codes and bpe_codes[pair] < min_rank:
                min_pair = pair
                min_rank = bpe_codes[pair]
        
        if min_pair is None:
            break

        # Merge
        new_symbol = ''.join(min_pair)
        i = 0
        while i < len(word) - 1:
            if word[i] == min_pair[0] and word[i+1] == min_pair[1]:
                word[i:i+2] = [new_symbol]
                break
            else:
                i += 1
        pairs = _get_pairs(word)
    return word


def split_text(text:str) -> list[str]:
    """
    Split the text into BPE tokens
    """
    tokens = []
    words = text.strip().split()
    for word in words:
        chars = list(word) + ['</w>']
        bpe_word = apply_bpe(chars)
        tokens.extend(bpe_word)
    return tokens

def encode_text(text:str) -> list[int]:
    """
    Encode the text into token ids
    """
    tokens = split_text(text)
    return [token_map[token] for token in tokens]

def decode(data: list[int]) -> str:
    """
    Decode the token ids into text
    """
    tokens = [inv_token_map[token] for token in data]
    words = []
    word = ''
    for token in tokens:
        if token == '</w>':
            words.extend(word)
            word = ''
        else:
            word += token.replace('</w>', '')
    
    if word:
        words.append(word)
    
    return ''.join(words)

In [19]:
tokens = encode_text(sentence)
print(tokens)

[0, 694, 429, 729, 1220, 1096, 1541, 912, 1217, 636, 674, 1256, 176, 639, 808, 622, 971, 464, 624, 1271, 464, 705, 1501, 674, 1010, 745, 449, 640, 634, 841, 603, 300, 1233, 860, 615, 1047, 736, 631, 703, 634, 1587, 1524, 611, 674, 602, 685, 477, 500, 870, 888, 847, 21, 999, 603, 1174, 941, 452, 1019, 737, 1034, 603, 293, 1398, 624, 1487, 735, 130, 656, 754, 452, 813, 619, 620, 678, 1289, 904, 693, 630, 937, 1116, 834, 1387, 610, 1541, 607, 1112, 616, 924, 674, 1099, 615, 1144, 828, 622, 349, 1328, 668, 477, 1150, 830, 679, 778, 603, 426, 37, 652, 1216, 646, 509, 815, 603, 732, 972, 601, 1034, 670, 603, 1174, 130, 1279, 943, 1057, 623, 1261, 631, 1189, 1083, 1195, 602, 429, 1140, 754, 610, 176, 575, 477, 641, 631, 674, 1308, 1179, 845, 610, 1443, 749, 652, 670, 1040, 338, 880, 674, 1030, 626, 349, 726, 748, 817, 1384, 603, 732, 1433, 1409, 880, 620, 626, 1590, 429, 635, 61, 960, 630, 801, 711, 626, 1590, 731, 940, 704, 622, 1294, 947, 630, 1473, 837, 647, 858, 1390, 659, 646, 711, 759, 

In [20]:
decoded_sentence = decode(tokens)
print(decoded_sentence)

</BOT>Thewetmarblefloorpressedonhischeeklikeathousandhandsslappinghisfacefrozenintime.Smatteringpissofrainignoredhisindignantmumblings.Hiseyesfluttered.Pinsandneedsranfromfingertoshoulderashepushedbackagainstthefloor,contortinghisachingbodyintoacrossleggedposition.Lastnightwasbad.Hegatheredthat.Hisroutinedullnessofthoughcreptinwardsfromtheedgesofhismindtowardtheblackmistthatveiledhismostrecentmemories.Hestruggledtorecallwhateverhecouldn'trecallbutonlyforamomentbeforehedecideditprobablywasn'tworththeeffort.Heglancedaroundtheroomforafewminutesbeforeconcludingthatheprobablydidn'tknowwherehewas.Hisinvestigationwasn'tentirelyfruitless,hediscoveredamostlyfullbottleofvodka.Itwascheapbutwoulddefinitelygetthejobdone.Takingafewswigsmadeitchildishlyeasytoignorethatgiganticblackcloudoffogblottingoutwhateverthehellhedidbeforehewokeup.Therewasamirrorintheroomandforwantofanythingmoreinterestingtostudyhegazedathimself.Itwasagamehe'dplaywithhimself,glancingatthemirrorandseeingifhecouldrecognizetheperso